In [ ]:
from typing import List


class Solution:

    def corpFlightBookings(self, bookings: List[List[int]], n: int) -> List[int]:
        # 区间+和 单点查询
        s = [0] * (2 << n.bit_length())
        lazy = [0] * (2 << n.bit_length())

        def pushdown(o, l, r, m):
            if (x := lazy[o]) != 0:
                s[o * 2] += (m - l + 1) * x
                s[o * 2 + 1] += (r - m) * x
                lazy[o * 2] += x
                lazy[o * 2 + 1] += x
                lazy[o] = 0

        def add(o, l, r, L, R, val):  # [L,R] + val
            if L <= l and r <= R:
                s[o] += (r - l + 1) * val
                lazy[o] += val
                return
            m = l + r >> 1
            pushdown(o, l, r, m)
            if m >= L:
                add(o * 2, l, m, L, R, val)
            if m + 1 <= R:
                add(o * 2 + 1, m + 1, r, L, R, val)
            s[o] = s[o * 2] + s[o * 2 + 1]

        def query(o, l, r, idx):  # 单点查询
            if l == r:
                return s[o]
            m = l + r >> 1
            pushdown(o, l, r, m)
            if m >= idx:
                return query(o * 2, l, m, idx)
            else:
                return query(o * 2 + 1, m + 1, r, idx)

        for x, y, seat in bookings:
            add(1, 0, n - 1, x - 1, y - 1, seat)

        return [query(1, 0, n - 1, i) for i in range(n)]

In [3]:
l = [1, 2, 3]
list(map(lambda x: x + 2, l))
[1, 2] + [3, 4]


[1, 2, 3, 4]

In [8]:
# 超时
class Node:

    def __init__(self, si, ei):
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.left = None
        self.right = None
        self.val = [0] * (ei - si + 1)
        self.lazy = 0


class SegmentTree:

    def __init__(self, si, ei):
        self.root = Node(si, ei)

    def pushDown(self, node: Node):
        if not node.left:
            node.left = Node(node.si, node.mi)
        if not node.right:
            node.right = Node(node.mi + 1, node.ei)
        if node.lazy == 0:
            return
        node.left.val = list(map(lambda x: x + node.lazy, node.left.val))
        node.right.val = list(map(lambda x: x + node.lazy, node.right.val))
        node.left.lazy += node.lazy
        node.right.lazy += node.lazy
        node.lazy = 0

    def update(self, node: Node, l, r, val):  # [l,r]值都+val
        if node.ei < l or node.si > r:
            return
        if l <= node.si <= node.ei <= r:
            node.val = list(map(lambda x: x + val, node.val))
            node.lazy += val
            return
        self.pushDown(node)
        if node.mi >= l:
            self.update(node.left, l, r, val)
        if node.mi + 1 <= r:
            self.update(node.right, l, r, val)
        node.val = node.left.val + node.right.val

    def query(self, node: Node, l, r):
        if node.ei < l or node.si > r:
            return 0
        if l <= node.si <= node.ei <= r:
            return node.val
        self.pushDown(node)
        ans = []
        if node.mi >= l:
            ans += self.query(node.left, l, r)
        if node.mi + 1 <= r:
            ans += self.query(node.right, l, r)
        return ans


from typing import List


class Solution:

    def corpFlightBookings(self, bookings: List[List[int]], n: int) -> List[int]:
        st = SegmentTree(1, n)
        for si, ei, seats in bookings:
            st.update(st.root, si, ei, seats)
        ans = st.query(st.root, 1, n)
        return ans


s = Solution()
# s.corpFlightBookings([[1, 2, 10], [2, 3, 20], [2, 5, 25]], 5)
s.corpFlightBookings([[5, 6, 45], [5, 6, 5], [1, 6, 10], [5, 6, 5], [1, 2, 15], [1, 2, 5]], 6)


[30, 30, 10, 10, 65, 65]

In [10]:
# 过了
class Node:

    def __init__(self, si, ei):
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.left = None
        self.right = None
        self.val = 0
        self.lazy = 0


class SegmentTree:

    def __init__(self, si, ei):
        self.root = Node(si, ei)

    def pushDown(self, node: Node):
        if not node.left:
            node.left = Node(node.si, node.mi)
        if not node.right:
            node.right = Node(node.mi + 1, node.ei)
        if node.lazy == 0:
            return
        node.left.val += node.lazy
        node.right.val += node.lazy
        node.left.lazy += node.lazy
        node.right.lazy += node.lazy
        node.lazy = 0

    def update(self, node: Node, l, r, val):  # [l,r]值都+val
        if node.ei < l or node.si > r:
            return
        if l <= node.si <= node.ei <= r:
            node.val += val
            node.lazy += val
            return
        self.pushDown(node)
        if node.mi >= l:
            self.update(node.left, l, r, val)
        if node.mi + 1 <= r:
            self.update(node.right, l, r, val)
        node.val = node.left.val + node.right.val

    def query(self, node: Node, l, r):
        if node.ei < l or node.si > r:
            return 0
        if l <= node.si <= node.ei <= r:
            return node.val
        self.pushDown(node)
        ans = 0
        if node.mi >= l:
            ans += self.query(node.left, l, r)
        if node.mi + 1 <= r:
            ans += self.query(node.right, l, r)
        return ans


from typing import List


class Solution:

    def corpFlightBookings(self, bookings: List[List[int]], n: int) -> List[int]:
        st = SegmentTree(1, n)
        for si, ei, seats in bookings:
            st.update(st.root, si, ei, seats)
        ans = []
        for i in range(1, n + 1):
            ans.append(st.query(st.root, i, i))
        return ans


s = Solution()
# s.corpFlightBookings([[1, 2, 10], [2, 3, 20], [2, 5, 25]], 5)
s.corpFlightBookings([[5, 6, 45], [5, 6, 5], [1, 6, 10], [5, 6, 5], [1, 2, 15], [1, 2, 5]], 6)


[30, 30, 10, 10, 65, 65]